In [5]:
import gc
import glob

import geopandas as gpd
import momepy as mm
import numpy as np
import pandas as pd
import shapely
from libpysal.graph import Graph, read_parquet
import datetime

In [6]:
regions_buildings_dir = '/data/uscuni-ulce/regions/buildings/'
buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'
overture_streets_dir = '/data/uscuni-ulce/overture_streets/'
streets_dir = '/data/uscuni-ulce/processed_data/streets/'
enclosures_dir = '/data/uscuni-ulce/processed_data/enclosures/'
tessellations_dir = '/data/uscuni-ulce/processed_data/tessellations/'
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
chars_dir = '/data/uscuni-ulce/processed_data/chars/'

regions_datadir = "/data/uscuni-ulce/"
eubucco_files = glob.glob(regions_datadir + "eubucco_raw/*")

In [7]:
# prague - 69333
# freiburg - 5883
# 4 - largest region

# df = pd.read_parquet('/data/uscuni-ulce/regions/cadastre_id_to_region.parquet')
# df.region.value_counts()
# '''
# region
# 4.0         11084489
# 107131.0     3078558
# 16242.0      1457482
# 4214.0       1389195
# 8707.0       1220411
#               ...   
# 7727.0         12682
# 6741.0         12235
# 10970.0        12170
# 10847.0        10871
# 15545.0        10409
# Name: count, Length: 828, dtype: int64



# df.region.value_counts().iloc[:20].index.values.astype(int)

# array([     4, 107131,  16242,   4214,   8707,  51289, 114311,  55763,
#         14679,  62954,  10579,  38679,  86873,  34266,     10,  80404,
#         18557,  69333,   1154, 130658])
# '''

In [40]:
region_name = 69333
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
    )
region_id, region_hull = region_hulls.loc[region_name].name, region_hulls.loc[region_name].convex_hull

In [47]:
for region_id, _ in region_hulls.iloc[1:].iterrows():
    buildings = gpd.read_parquet(buildings_dir + f'buildings_{region_id}.parquet')
    if ('index' in buildings.columns) or ('level_0' in buildings.columns):
        print(region_id)
    

10
132
134
286
313
400
523
765
801
832
913
960
1124
1154
1387
1478
1515
1605
1718
1736
1782
1970
1981
2096
2322
2350
2478
2514
2625
2728
2975
3039
3109
3150
3221
3250
3526
3610
3612
3701
3705
3752
3759
3981
4070
4214
4215
4235
4284
4356
4382
4723
4805
5096
5191
5246
5310
5408
5427
5662
5671
5766
5883
6254
6529
6560
6576
6741
6749
6811
6873
6996
7068
7094
7280
7485
7528
7534
7681
7688
7712
7727
7805
7914
7937
7963
8046
8216
8238
8256
8265
8345
8374
8396
8592
8707
8731
8757
8759
8813
9016
9064
9074
9150
9169
9194
9284
9824
9924
9954


KeyboardInterrupt: 

In [34]:
region_hulls.shape

(828, 1)

## Buildings


In [28]:
from core.generate_buildings import read_region_buildings, process_region_buildings

In [41]:
%%time

buildings = gpd.read_parquet(regions_buildings_dir + f'buildings_{region_id}.pq')

CPU times: user 252 ms, sys: 77.8 ms, total: 330 ms
Wall time: 307 ms


'/data/uscuni-ulce/processed_data/buildings/'

In [31]:
%%time

buildings = process_region_buildings(buildings, True, simplification_tolerance=.1, merge_limit=25)

Percent polygons:  1.0
Final polygons:  457127 , dropped:  0.02918639964321368
CPU times: user 1min 17s, sys: 0 ns, total: 1min 17s
Wall time: 1min 17s


In [10]:

# orig_buildings = gpd.read_parquet(regions_buildings_dir + f'buildings_{region_id}.pq')
# dropped_buildings = orig_buildings[~orig_buildings['iid'].isin(buildings['iid'])]


# m = dropped_buildings.explore(color='red')
# m = buildings.explore(m=m)
# m

In [11]:
buildings.to_parquet(buildings_dir + f"buildings_{region_id}.parquet")

## Streets

In [6]:
from core.generate_streets import process_region_streets, process_single_region_streets

In [7]:
# from core.utils import largest_regions
# region_hulls = region_hulls[~region_hulls.index.isin(largest_regions)]

In [7]:
processed_region_ids = [int(s.split('.')[0].split('_')[-1])for s in glob.glob(streets_dir + '*')]
region_hulls = region_hulls[~region_hulls.index.isin(processed_region_ids)]
region_hulls

,convex_hull
labels,


In [8]:
%%time
## processs streets
streets = process_region_streets(region_id, overture_streets_dir, buildings_dir)

/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/simplify.py:343: UserWarning: An error occured at location POINT (4634734.833449237 3007874.1868347814). The artifact has not been simplified. The original message:
can only convert an array of size 1 to a Python scalar
  roads_cleaned = simplify_singletons(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/simplify.py:200: UserWarning: Could not create a connectio

CPU times: user 9min 39s, sys: 3.21 s, total: 9min 42s
Wall time: 9min 36s


In [11]:
## save streets
streets.to_parquet(streets_dir + f'streets_{region_id}.parquet')

In [47]:
buildings.iloc[np.unique(building_idxs)].to_parquet('ws_buildings.pq')

In [48]:
streets.iloc[np.unique(idxs)].to_parquet('ws_streets.pq')

In [10]:
# streets.explore(tiles='cartodbpositron', prefer_canvas=True)

In [19]:
# import geopandas as gpd
# gpd.read_parquet(streets_dir + f'streets_{region_id}.parquet').explore()

In [20]:
# gpd.read_parquet(enclosures_dir + f'enclosure_{region_id}.parquet').explore()

## Enclosures & Tessellation

In [10]:
from core.generate_elements import process_region_elements, generate_enclosures_representative_points, generate_tess

In [11]:
# 21894
# 47090

In [13]:
# %%time
enclosures, tesselations = process_region_elements(buildings_dir, streets_dir, region_id)

---- Processing region:  69333 2024-10-11 13:37:46.843847


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Retrying tesselation with less buildings, potentially changing building data.


ValueError: cannot insert level_0, already exists

,level_0,index,currentUse,buildingNature,heightAboveGround,iid,geometry
0,1,1,None,None,NaN,80677518,"POLYGON ((4617480.926 3025927.323, 4617486.098..."
1,2,2,None,None,NaN,80677519,"POLYGON ((4617331.885 3025855.863, 4617331.941..."
2,3,3,None,None,NaN,80677520,"POLYGON ((4617495.899 3025921.454, 4617499.35 ..."
3,4,4,None,None,NaN,80677521,"POLYGON ((4617561.862 3026075.057, 4617564.522..."
4,5,5,None,None,NaN,80677522,"POLYGON ((4617511.527 3025994.104, 4617518.733..."
...,...,...,...,...,...,...,...
455057,457120,457122,None,None,NaN,84935651,"POLYGON ((4632982.934 2965305.432, 4632984.67 ..."
455058,457121,457123,None,None,NaN,84935653,"POLYGON ((4630241.975 2968160.071, 4630245.22 ..."
455059,457122,457124,None,None,NaN,84935654,"POLYGON ((4632630.507 2966988.048, 4632632.381..."
455060,457123,457125,None,None,NaN,84935655,"POLYGON ((4633011.979 2966889.52, 4633025.38 2..."


In [14]:
buildings = gpd.read_parquet(
    buildings_dir + f"buildings_{region_id}.parquet"
)
streets = gpd.read_parquet(streets_dir + f"streets_{region_id}.parquet")
enclosures = generate_enclosures_representative_points(buildings, streets)

In [18]:
%%time
tesselations = generate_tess(buildings.make_valid().normalize(),
                             enclosures.make_valid().normalize(),
                             n_workers=-1)
tesselation_coverage = np.isin(
    buildings.index.values, tesselations.index.values
)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


CPU times: user 2min 8s, sys: 1.81 s, total: 2min 10s
Wall time: 3min 47s


In [19]:
not tesselation_coverage.all()

True

In [20]:
num_problem_buildings = (~tesselation_coverage).sum()
buildings = buildings[tesselation_coverage].reset_index()

ValueError: cannot insert level_0, already exists

In [25]:
buildings

,level_0,index,currentUse,buildingNature,heightAboveGround,iid,geometry
0,1,1,None,None,NaN,80677518,"POLYGON ((4617480.926 3025927.323, 4617486.098..."
1,2,2,None,None,NaN,80677519,"POLYGON ((4617331.885 3025855.863, 4617331.941..."
2,3,3,None,None,NaN,80677520,"POLYGON ((4617495.899 3025921.454, 4617499.35 ..."
3,4,4,None,None,NaN,80677521,"POLYGON ((4617561.862 3026075.057, 4617564.522..."
4,5,5,None,None,NaN,80677522,"POLYGON ((4617511.527 3025994.104, 4617518.733..."
...,...,...,...,...,...,...,...
455057,457120,457122,None,None,NaN,84935651,"POLYGON ((4632982.934 2965305.432, 4632984.67 ..."
455058,457121,457123,None,None,NaN,84935653,"POLYGON ((4630241.975 2968160.071, 4630245.22 ..."
455059,457122,457124,None,None,NaN,84935654,"POLYGON ((4632630.507 2966988.048, 4632632.381..."
455060,457123,457125,None,None,NaN,84935655,"POLYGON ((4633011.979 2966889.52, 4633025.38 2..."


In [24]:
buildings[tesselation_coverage]

,level_0,index,currentUse,buildingNature,heightAboveGround,iid,geometry
0,1,1,None,None,NaN,80677518,"POLYGON ((4617480.926 3025927.323, 4617486.098..."
1,2,2,None,None,NaN,80677519,"POLYGON ((4617331.885 3025855.863, 4617331.941..."
2,3,3,None,None,NaN,80677520,"POLYGON ((4617495.899 3025921.454, 4617499.35 ..."
3,4,4,None,None,NaN,80677521,"POLYGON ((4617561.862 3026075.057, 4617564.522..."
4,5,5,None,None,NaN,80677522,"POLYGON ((4617511.527 3025994.104, 4617518.733..."
...,...,...,...,...,...,...,...
455057,457120,457122,None,None,NaN,84935651,"POLYGON ((4632982.934 2965305.432, 4632984.67 ..."
455058,457121,457123,None,None,NaN,84935653,"POLYGON ((4630241.975 2968160.071, 4630245.22 ..."
455059,457122,457124,None,None,NaN,84935654,"POLYGON ((4632630.507 2966988.048, 4632632.381..."
455060,457123,457125,None,None,NaN,84935655,"POLYGON ((4633011.979 2966889.52, 4633025.38 2..."


In [ ]:
# from shapely import Point
# from shapely import GEOSException
# tesselations = None

# try:

#     tesselations = generate_tess(buildings, enclosures, n_workers=-1)

# except GEOSException as e:
#     txt = str(e)
#     problem_point_coords = [float(s) for s in txt[45:].split('. T')[0].split(' ')]
#     problem_point = Point(*problem_point_coords)
#     problem_building = buildings.iloc[buildings.sindex.query(problem_point, predicate='intersects')]
#     buildings = buildings.drop(buildings.sindex.query(problem_point, predicate='intersects'))
#     tesselations = generate_tess(buildings, enclosures, n_workers=-1)

In [ ]:
# new_buildings = gpd.read_parquet(buildings_dir + f'buildings_{region_id}.parquet')

# # m = buildings.explore(max_zoom=20)
# # m = new_buildings.explore(m=m, color='red')


# # m = buildings[~buildings['iid'].isin(new_buildings['iid'])].explore(m=m, color='red')

# # m = dropped_buildings.explore(m=m, color='red')
# # m = enclosures.explore(m=m, color='green')
# # m = tesselations[tesselations['enclosure_index'] == 211].reset_index().explore(m=m, color='green')
# # m

In [ ]:
enclosures.to_parquet(enclosures_dir + f"enclosure_{region_id}.parquet")
print("Processed enclosures")

## save files
tesselations.to_parquet(
    tessellations_dir + f"tessellation_{region_id}.parquet"
)
print("processed tesselations")

In [134]:
# import lonboard
# layer = lonboard.PolygonLayer.from_geopandas(tesselations, opacity=0.15)
# m = lonboard.Map([layer])
# m

In [22]:
# layer = lonboard.PolygonLayer.from_geopandas(enclosures, opacity=0.15)
# m = lonboard.Map([layer])
# m

## Graphs

In [25]:
from core.generate_ngraphs import process_region_graphs

In [26]:
%%time
process_region_graphs(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
)

Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 243 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


Built nodes graph knn=1
CPU times: user 42.4 s, sys: 888 ms, total: 43.3 s
Wall time: 43.2 s


In [141]:
processed_region_ids = [int(s.split('.')[0].split('_')[-1])for s in glob.glob(graph_dir + '*')]
region_hulls = region_hulls[~region_hulls.index.isin(processed_region_ids)]
region_hulls

In [ ]:
from joblib import Parallel, delayed
n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(process_region_graphs)(     region_id,
                                        graph_dir,
                                        buildings_dir,
                                        streets_dir,
                                        enclosures_dir,
                                        tessellations_dir
                                  ) 
    for region_id, _ in region_hulls.iterrows())

## Characters

In [27]:
from core.generate_chars import process_single_region_chars, process_street_chars

In [28]:
%%time
process_single_region_chars(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
    chars_dir
)

2024-10-10 16:50:14.663553 ----Processing ------ 69333
Processing streets
Processing enclosures
Processing buildings


/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:576: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(90 - degrees[true_angles]))
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:737: RuntimeWarning: Mean of empty slice
  return Series({"mean": np.nanmean(dists), "std": np.nanstd(dists)})
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Processing tessellation
CPU times: user 10min 17s, sys: 4.32 s, total: 10min 21s
Wall time: 10min 19s


## Generate primary data

In [29]:
tessellation = gpd.read_parquet(chars_dir + f"tessellations_chars_{region_id}.parquet")
buildings = gpd.read_parquet(chars_dir + f"buildings_chars_{region_id}.parquet")
enclosures = gpd.read_parquet(chars_dir + f"enclosures_chars_{region_id}.parquet")
streets = gpd.read_parquet(chars_dir + f"streets_chars_{region_id}.parquet")
nodes = gpd.read_parquet(chars_dir + f"nodes_chars_{region_id}.parquet")

In [30]:

merged = pd.merge(
    tessellation.drop(columns=["geometry"]),
    buildings.drop(columns=["nodeID", "geometry", 'nID']),
    right_index=True,
    left_index=True,
    how="left",
)

merged = merged.merge(
    enclosures.drop(columns="geometry"),
    right_on="eID",
    left_on="enclosure_index",
    how="left",
)

merged = merged.merge(streets.drop(columns="geometry"), on="nID", how="left")
merged = merged.merge(nodes.drop(columns="geometry"), on="nodeID", how="left")

merged = merged.drop(
    columns=[
        "nID",
        "eID",
        "nodeID",
        "mm_len",
        "cdsbool",
        "node_start",
        "node_end",
        "x",
        "y",
        "enclosure_index",
        # "id",
        # "osm_id",
        "index",  ## maybe keep
    ]
)
merged = merged.set_index(tessellation.index)

In [31]:
from core.utils import used_keys

In [32]:
primary = merged[list(used_keys.keys())]
primary.shape

(459980, 63)

In [33]:
primary.to_parquet(chars_dir + f'primary_chars_{region_id}.parquet')